In [2]:
# https://www.freemaptools.com/elevation-finder.htm
# https://gis.stackexchange.com/questions/338392/getting-elevation-for-multiple-lat-long-coordinates-in-python

import requests
import urllib
import pandas as pd

In [3]:
stations = pd.read_csv("../Data/weather/stations.tsv", delimiter="\t")

In [4]:
stations.head()

,WBAN,COUNTRY,STATE,LOCATION,VECTOR,NAME,LATITUDE,LONGITUDE,ELEVATION,STATUS,COMMISSIONING,CLOSING,OPERATION,PAIRING,NETWORK,STATION_ID
0,03047,US,TX,Monahans,6 ENE,Sandhills State Park,31.62,-102.80,2724,Commissioned,2004-01-11 19:00:00.0,NaN,Operational,NaN,USCRN,1019
1,03048,US,NM,Socorro,20 N,Sevilleta National Wildlife Refuge (LTER Site),34.35,-106.88,4847,Commissioned,2004-01-11 19:00:00.0,NaN,Operational,NaN,USCRN,1020
2,03054,US,TX,Muleshoe,19 S,Muleshoe National Wildlife Refuge (Headquarter...,33.95,-102.77,3742,Commissioned,2004-04-22 20:00:00.0,NaN,Operational,NaN,USCRN,1067
3,03055,US,OK,Goodwell,2 E,OK Panhandle Research & Extn. Center (Native ...,36.59,-101.59,3266,Commissioned,2004-04-22 20:00:00.0,NaN,Operational,NaN,USCRN,1068
4,03060,US,CO,Montrose,11 ENE,Black Canyon of the Gunnison National Park (Ve...,38.54,-107.69,8402,Commissioned,2004-09-07 20:00:00.0,NaN,Operational,NaN,USCRN,1109


In [7]:
# USGS Elevation Point Query Service
# https://nationalmap.gov/epqs/
url = r"https://nationalmap.gov/epqs/pqs.php?"


def elevation_function(df, lat_column, lon_column, ele_column):
    """Query service using lat, lon. add the elevation values as a new column."""
    elevations = []
    for lat, lon in zip(df[lat_column], df[lon_column]):

        # define rest query params
        params = {"output": "json", "x": lon, "y": lat, "units": "Meters"}

        # format query string and return query value
        result = requests.get((url + urllib.parse.urlencode(params)))
        elevations.append(
            result.json()["USGS_Elevation_Point_Query_Service"]["Elevation_Query"][
                "Elevation"
            ]
        )

    df[ele_column] = elevations


elevation_function(stations, "LATITUDE", "LONGITUDE", "ELEVATION")
stations.head()

,WBAN,COUNTRY,STATE,LOCATION,VECTOR,NAME,LATITUDE,LONGITUDE,ELEVATION,STATUS,COMMISSIONING,CLOSING,OPERATION,PAIRING,NETWORK,STATION_ID
0,03047,US,TX,Monahans,6 ENE,Sandhills State Park,31.62,-102.80,829.32,Commissioned,2004-01-11 19:00:00.0,NaN,Operational,NaN,USCRN,1019
1,03048,US,NM,Socorro,20 N,Sevilleta National Wildlife Refuge (LTER Site),34.35,-106.88,1470.51,Commissioned,2004-01-11 19:00:00.0,NaN,Operational,NaN,USCRN,1020
2,03054,US,TX,Muleshoe,19 S,Muleshoe National Wildlife Refuge (Headquarter...,33.95,-102.77,1137.76,Commissioned,2004-04-22 20:00:00.0,NaN,Operational,NaN,USCRN,1067
3,03055,US,OK,Goodwell,2 E,OK Panhandle Research & Extn. Center (Native ...,36.59,-101.59,990.17,Commissioned,2004-04-22 20:00:00.0,NaN,Operational,NaN,USCRN,1068
4,03060,US,CO,Montrose,11 ENE,Black Canyon of the Gunnison National Park (Ve...,38.54,-107.69,2560.33,Commissioned,2004-09-07 20:00:00.0,NaN,Operational,NaN,USCRN,1109


In [11]:
stations[["WBAN", "LATITUDE", "LONGITUDE", "ELEVATION"]].to_pickle(
    "../Data/weather/Elevation.bz2"
)